**Finding Similar Movie**

We will start by loading MovieLens dataset from scikit-suprise


In [1]:
!pip install scikit-surprise
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505171 sha256=8c957d6c70ed6e864774e087b84ce39c55ff1190448684a2e9d19965a23ed965
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [2]:
# Load the built-in MovieLens-100k dataset
data = Dataset.load_builtin('ml-100k')
import os
from surprise import get_dataset_dir
file_path = os.path.join(get_dataset_dir(), 'ml-100k', 'ml-100k', 'u.data')
print(file_path)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
/root/.surprise_data/ml-100k/ml-100k/u.data


In [3]:
import pandas as pd
from surprise import Dataset, Reader, get_dataset_dir
import os

# 1. Load the ratings data:
file_path = os.path.join(get_dataset_dir(), 'ml-100k', 'ml-100k', 'u.data')
reader = Reader(line_format='user item rating timestamp', sep='\t', rating_scale=(1, 5))
data = Dataset.load_from_file(file_path, reader=reader)
ratings_df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'movie_id', 'rating', 'timestamp'])

# Convert 'movie_id' to numeric in ratings_df:
ratings_df['movie_id'] = pd.to_numeric(ratings_df['movie_id'])


# 2. Load the movie titles data:
movies_file_path = os.path.join(get_dataset_dir(), 'ml-100k', 'ml-100k', 'u.item')
movies_df = pd.read_csv(movies_file_path, sep='|', encoding='latin-1', header=None,
                        usecols=[0, 1], names=['movie_id', 'title'])

# 3. Merge the two DataFrames:
ratings_df = pd.merge(ratings_df, movies_df, on='movie_id')

# Now you have a DataFrame with 'user_id', 'movie_id', 'rating', 'timestamp', and 'title' columns
ratings_df.head()

,user_id,movie_id,rating,timestamp,title
0,196,242,3.0,881250949,Kolya (1996)
1,186,302,3.0,891717742,L.A. Confidential (1997)
2,22,377,1.0,878887116,Heavyweights (1994)
3,244,51,2.0,880606923,Legends of the Fall (1994)
4,166,346,1.0,886397596,Jackie Brown (1997)


pivot table function on dataframe will construct user/movie rating matrix. NaN indicates specific user didnt rate


In [4]:
movieRating = ratings_df.pivot_table(index=['user_id'],columns=['title'],values='rating')
movieRating.head()

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
10,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN


Lets extract a series ofuser who rated movie_id 10

In [5]:
angrymanRating  = movieRating['12 Angry Men (1957)']
angrymanRating.head()

,12 Angry Men (1957)
user_id,
1,5.0
10,5.0
100,NaN
101,NaN
102,NaN


In [7]:
# movie_Id1 = movieRating['1']
# movie_Id1.head()

Compute the pairwise correlation of 12 angry man's vector of user rating with every other movie.. After that we will drop any result that have no data and construct a new Dataframe of Movies and their correlation score (similarity) to 12 angry Man's.

In [8]:
similarMovies = movieRating.corrwith(angrymanRating)
similarMovies = similarMovies.dropna()
df = pd.DataFrame(similarMovies)
df.head(10)

/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.11/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,0
title,
'Til There Was You (1997),-0.500000
101 Dalmatians (1996),-0.049890
12 Angry Men (1957),1.000000
187 (1997),0.666667
2 Days in the Valley (1996),0.256625
"20,000 Leagues Under the Sea (1954)",0.274772
2001: A Space Odyssey (1968),0.178848
"39 Steps, The (1935)",0.457176
8 1/2 (1963),0.635698


Lets sort by similarity score and we should have the movies most similar to 12 angry Man

In [9]:
similarMovies = similarMovies.sort_values(ascending=False)
similarMovies.head(10)

,0
title,
Dingo (1992),1.0
Nina Takes a Lover (1994),1.0
Surviving Picasso (1996),1.0
American Dream (1990),1.0
Grace of My Heart (1996),1.0
Dream With the Fishes (1997),1.0
FairyTale: A True Story (1997),1.0
Double Happiness (1994),1.0
Gridlock'd (1997),1.0


In [10]:
!git clone https://github.com/samundra-9/Machine_Learning.git


Cloning into 'Machine_Learning'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 8 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (8/8), 6.00 KiB | 6.00 MiB/s, done.


In [11]:
%cd Machine_Learning/MovieRecommendationSystem

/content/Machine_Learning/MovieRecommendationSystem


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [13]:
import shutil
shutil.copy("/content/MovieCosineSimilarity.ipynb", "/content/Machine_Learning/MovieRecommendationSystem/")

FileNotFoundError: [Errno 2] No such file or directory: '/content/MovieCosineSimilarity.ipynb'